In [2]:
import pandas as pd 
import re
import math
import os
import sys
import cv2
import glob
import time
import imutils
import numpy as np
from functools import reduce

In [3]:
from moviepy.tools import subprocess_call
from moviepy.config import get_setting

def ffmpeg_extract_subclip(filename, t1, t2, targetname=None):
    """ Makes a new video file playing video file ``filename`` between
    the times ``t1`` and ``t2``. """
    name, ext = os.path.splitext(filename)
    if not targetname:
        T1, T2 = [int(1000*t) for t in [t1, t2]]
        targetname = "%sSUB%d_%d.%s" % (name, T1, T2, ext)

    cmd = [get_setting("FFMPEG_BINARY"),"-y",
           "-ss", "%0.2f"%t1,
           "-i", filename,
           "-t", "%0.2f"%(t2-t1),
           "-vcodec", "copy", "-acodec", "copy", targetname]

    subprocess_call(cmd)

In [4]:
video_info_path = 'C:/Users/Seogki/Downloads/0819 UCF 데이터 재라벨링 - 시트1.csv'
#video_path = ''

In [5]:
df = pd.read_csv(video_info_path)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   filename    102 non-null    object
 1   fight       62 non-null     object
 2   nofight     57 non-null     object
 3   child       11 non-null     object
 4   info        37 non-null     object
 5   Unnamed: 5  52 non-null     object
dtypes: object(6)
memory usage: 5.0+ KB


In [7]:
df.drop('Unnamed: 5', axis=1, inplace=True)

In [8]:
x_df = df[df['info'].map(lambda x: str(x).startswith('x'))]

In [9]:
child_df = df[df['child'].map(lambda x: str(x).startswith('o'))]

In [10]:
no_x_df = df[df['info'].map(lambda x: not str(x).startswith('x'))]

In [11]:
no_x_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79 entries, 0 to 102
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  78 non-null     object
 1   fight     62 non-null     object
 2   nofight   57 non-null     object
 3   child     10 non-null     object
 4   info      13 non-null     object
dtypes: object(5)
memory usage: 3.7+ KB


In [12]:
x_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24 entries, 1 to 93
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  24 non-null     object
 1   fight     0 non-null      object
 2   nofight   0 non-null      object
 3   child     1 non-null      object
 4   info      24 non-null     object
dtypes: object(5)
memory usage: 1.1+ KB


In [13]:
crop_df = no_x_df[no_x_df.child.map(lambda x: not str(x).startswith('o'))]

In [14]:
crop_df

,filename,fight,nofight,child,info
0,Abuse001,"(0:08,0:11)","(0:00,0:08)(0:11,0:25)",NaN,NaN
2,Abuse003,NaN,"(0:00,0:30)(0:36,1:07)(1:15,1:30)",NaN,NaN
3,Abuse004,"(0:30,0:34)","(0:00,0:16)(0:18,0:28)(1:08,1:30)(8:50,9:19)",NaN,NaN
4,Abuse005,"(0:24,0:29)\r","(0:00,0:21)",NaN,NaN
5,Abuse006,"(0:10,0:16)(0:32,0:37)","(0:00,0:09)(1:05,1:40)",NaN,NaN
...,...,...,...,...,...
98,Assault048,"(0:06,0:08)","(0:00,0:05)(0:11,1:00)",NaN,NaN
99,Assault049,"(0:11,0:17)","(0:00,0:10)",NaN,반복
100,Assault050,"(0:02,0:19)",NaN,NaN,NaN
101,Assault051,"(0:08,0:15)",NaN,NaN,NaN


In [15]:
crop_df.iloc[15:25]

,filename,fight,nofight,child,info
38,Abuse039,"(6:19,6:46)","(0:00,6:18)",NaN,NaN
39,Abuse040,"(0:37,0:39)(0:40,0:42)(0:47,0:49)(0:53,0:55)(1...","(0:11,0:36)",NaN,NaN
44,Abuse045,"(0:13,0:17)(0:29,0:36)(0:44,0:46)(0:55,0:57)(1...","(1:00,1:15)(1:30,2:09)",NaN,NaN
49,Abuse050,"(0:51, 1:00)(1:13,1:25)(1:31,1:41)(1:51,2:35)","(0:00,0:08)(1:04,1:12)",NaN,NaN
50,NaN,NaN,NaN,NaN,NaN
51,Assault001,"(0:04,0:18)(0:20,0:24)(0:50,1:01)(1:03,1:05)",NaN,NaN,NaN
52,Assault002,NaN,"(0:00,0:06)(1:03,1:12)(1:17,1:24)",NaN,NaN
53,Assault003,"(0:12,0:26)(0:33,0:46)(1:15,1:18)","(0:00,0:09)(1:22,1:29)",NaN,NaN
54,Assault004,"(0:29,0:41)(0:46,1:08)(1:22,1:28)","(0:00,0:11)(0:12,0:24)(1:29,1:45)",NaN,NaN
55,Assault005,"(0:15,0:23)","(0:00,0:12)",NaN,NaN


In [16]:
crop_df.drop(50, inplace=True)

C:\Users\Seogki\Anaconda3\lib\site-packages\pandas\core\frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [17]:
crop_df[15:25]

,filename,fight,nofight,child,info
38,Abuse039,"(6:19,6:46)","(0:00,6:18)",NaN,NaN
39,Abuse040,"(0:37,0:39)(0:40,0:42)(0:47,0:49)(0:53,0:55)(1...","(0:11,0:36)",NaN,NaN
44,Abuse045,"(0:13,0:17)(0:29,0:36)(0:44,0:46)(0:55,0:57)(1...","(1:00,1:15)(1:30,2:09)",NaN,NaN
49,Abuse050,"(0:51, 1:00)(1:13,1:25)(1:31,1:41)(1:51,2:35)","(0:00,0:08)(1:04,1:12)",NaN,NaN
51,Assault001,"(0:04,0:18)(0:20,0:24)(0:50,1:01)(1:03,1:05)",NaN,NaN,NaN
52,Assault002,NaN,"(0:00,0:06)(1:03,1:12)(1:17,1:24)",NaN,NaN
53,Assault003,"(0:12,0:26)(0:33,0:46)(1:15,1:18)","(0:00,0:09)(1:22,1:29)",NaN,NaN
54,Assault004,"(0:29,0:41)(0:46,1:08)(1:22,1:28)","(0:00,0:11)(0:12,0:24)(1:29,1:45)",NaN,NaN
55,Assault005,"(0:15,0:23)","(0:00,0:12)",NaN,NaN
56,Assault006,"(0:46,0:55)(1:40,1:42)(1:45,1:47)","(0:03,0:25)(2:41,2:44)(4:17,4:20)",NaN,NaN


In [18]:
crop_df.drop(columns=['child','info'],inplace=True)

C:\Users\Seogki\Anaconda3\lib\site-packages\pandas\core\frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [19]:
crop_df

,filename,fight,nofight
0,Abuse001,"(0:08,0:11)","(0:00,0:08)(0:11,0:25)"
2,Abuse003,NaN,"(0:00,0:30)(0:36,1:07)(1:15,1:30)"
3,Abuse004,"(0:30,0:34)","(0:00,0:16)(0:18,0:28)(1:08,1:30)(8:50,9:19)"
4,Abuse005,"(0:24,0:29)\r","(0:00,0:21)"
5,Abuse006,"(0:10,0:16)(0:32,0:37)","(0:00,0:09)(1:05,1:40)"
...,...,...,...
98,Assault048,"(0:06,0:08)","(0:00,0:05)(0:11,1:00)"
99,Assault049,"(0:11,0:17)","(0:00,0:10)"
100,Assault050,"(0:02,0:19)",NaN
101,Assault051,"(0:08,0:15)",NaN


In [20]:
crop_df.fillna('x',inplace=True)

C:\Users\Seogki\Anaconda3\lib\site-packages\pandas\core\frame.py:4459: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [21]:
crop_df[40:45]

,filename,fight,nofight
73,Assault023,x,"(0:11,0:!6)"
74,Assault024,"(0:00,0:03)",x
75,Assault025,"(0:13,0:15)(0:!5,0:17)","(0:05,0:09)"
76,Assault026,"(0:12,0:16)",x
77,Assault027,"(1:53,1:59)(2:49,2:52)","(0:00,1:18)(1:30,1:50)(2:00,2:48)(2:54,3:51)"


In [22]:
def time_to_sec(time):
    start = re.find(r'\((\d+),',time)
    end = re.find(r',(\d+)\),',time)
    print(start,end)
    return [sum(np.array([60,1]) * np.array(start.split(':'), dtype=np.int8)), sum(np.array([60,1]) * np.array(end.split(':'), dtype=np.int8))]

In [23]:
for item in crop_df.iterrows():
    fi = item[1].fight
    nofi = item[1].nofight
    print(item[1].filename)
    if fi == 'x' and nofi == 'x':
        print(item)
    regex = r'(\(\d:\d{2}.\d:\d{2}\))'
    print('--------------')
    fi_time = re.findall(regex,fi)
    print('fi time', fi_time)
    nofi_time = re.findall(regex,nofi)
    print('no fi time', nofi_time)
    print(re.findall(r'\(([\d ]+:[\d ]+),([\d ]+:[\d ]+)\)',fi))
    print(re.findall(r'\(([\d ]+:[\d ]+),([\d ]+:[\d ]+)\)',nofi))

Abuse001
--------------
fi time ['(0:08,0:11)']
no fi time ['(0:00,0:08)', '(0:11,0:25)']
[('0:08', '0:11')]
[('0:00', '0:08'), ('0:11', '0:25')]
Abuse003
--------------
fi time []
no fi time ['(0:00,0:30)', '(0:36,1:07)', '(1:15,1:30)']
[]
[('0:00', '0:30'), ('0:36', '1:07'), ('1:15', '1:30')]
Abuse004
--------------
fi time ['(0:30,0:34)']
no fi time ['(0:00,0:16)', '(0:18,0:28)', '(1:08,1:30)', '(8:50,9:19)']
[('0:30', '0:34')]
[('0:00', '0:16'), ('0:18', '0:28'), ('1:08', '1:30'), ('8:50', '9:19')]
Abuse005
--------------
fi time ['(0:24,0:29)']
no fi time ['(0:00,0:21)']
[('0:24', '0:29')]
[('0:00', '0:21')]
Abuse006
--------------
fi time ['(0:10,0:16)', '(0:32,0:37)']
no fi time ['(0:00,0:09)', '(1:05,1:40)']
[('0:10', '0:16'), ('0:32', '0:37')]
[('0:00', '0:09'), ('1:05', '1:40')]
Abuse007
--------------
fi time ['(0:02,0:05)']
no fi time ['(0:10,0:38)']
[('0:02', '0:05')]
[('0:10', '0:38')]
Abuse008
--------------
fi time ['(2:27,2:30)']
no fi time ['(0:00,0:10)', '(0:53,2:25)

In [31]:
full_vid= 'C:/Users/Seogki/Downloads/Anomaly-Videos-Part-1_Abuse/'
fight_path = 'C:/Users/Seogki/Downloads/UCF_Cropped/fight/' 
c_fight_path = 'C:/Users/Seogki/Downloads/UCF_Cropped/cropped_fight/' 
nofight_path = 'C:/Users/Seogki/Downloads/UCF_Cropped/noFight/' 
c_nofight_path = 'C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/' 

fight_header = 'AA-V-UCF-'
nofight_header = 'AA-N-UCF-'

vid_list = glob.glob(full_vid + '**/*.mp4', recursive=True)

In [40]:
pattern = r'\(([\d ]+:[\d ]+),([\d ]+:[\d ]+)\)'
out_path = []
for item in crop_df.iterrows():
    #print(item[1].filename)
    video = None
    for vid in vid_list:
        if item[1].filename in vid:
            video = vid
    
    source = video
    print('-----------------\n',video,'\n-------------------')
    #print(item[1].fight, item[1].nofight)
    dst_name = os.path.basename(video).split('.mp4')[0]
    # fight
    index = 1
    for start, end in re.findall(pattern,item[1].fight):
        start = float(start.split(':')[0]) * 60 + float(start.split(':')[1])
        end = float(end.split(':')[0]) * 60 + float(end.split(':')[1])
        print(start,end)
        assert end > start
        dst = fight_path + fight_header + dst_name + '_' + str(index)+ '.mp4'
        out_path.append(dst)
        ffmpeg_extract_subclip(source, int(start), int(end), targetname=dst)
        #print(dst)
        index += 1
    index = 1
    # nofight
    for start, end in re.findall(pattern,item[1].nofight):
        start = float(start.split(':')[0]) * 60 + float(start.split(':')[1])
        end = float(end.split(':')[0]) * 60 + float(end.split(':')[1])
        print(start, end)
        assert end > start
        dst = nofight_path + nofight_header + dst_name + '_' + str(index)+ '.mp4'
        out_path.append(dst)
        #print(dst)
        ffmpeg_extract_subclip(source, int(start), int(end), targetname=dst)
        index += 1


-----------------
 C:/Users/Seogki/Downloads/Anomaly-Videos-Part-1_Abuse\Abuse001_x264.mp4 
-------------------
8.0 11.0
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
0.0 8.0
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
11.0 25.0
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
-----------------
 C:/Users/Seogki/Downloads/Anomaly-Videos-Part-1_Abuse\Abuse003_x264.mp4 
-------------------
0.0 30.0
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
36.0 67.0
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
75.0 90.0
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
-----------------
 C:/Users/Seogki/Downloads/Anomaly-Videos-Part-1_Abuse\Abuse004_x264.mp4 
-------------------
30.0 34.0
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
0.0 16.0
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
18.0 28.0
Moviepy - Running:
>>> "+ "

Moviepy - Command successful
64.0 72.0
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
-----------------
 C:/Users/Seogki/Downloads/Anomaly-Videos-Part-1_Abuse\Assault001_x264.mp4 
-------------------
4.0 18.0
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
20.0 24.0
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
50.0 61.0
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
63.0 65.0
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
-----------------
 C:/Users/Seogki/Downloads/Anomaly-Videos-Part-1_Abuse\Assault002_x264.mp4 
-------------------
0.0 6.0
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
63.0 72.0
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
77.0 84.0
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
-----------------
 C:/Users/Seogki/Downloads/Anomaly-Videos-Part-1_Abuse\Assault003_x264.mp4 
-------------------


Moviepy - Command successful
-----------------
 C:/Users/Seogki/Downloads/Anomaly-Videos-Part-1_Abuse\Assault025_x264.mp4 
-------------------
13.0 15.0
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
5.0 9.0
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
-----------------
 C:/Users/Seogki/Downloads/Anomaly-Videos-Part-1_Abuse\Assault026_x264.mp4 
-------------------
12.0 16.0
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
-----------------
 C:/Users/Seogki/Downloads/Anomaly-Videos-Part-1_Abuse\Assault027_x264.mp4 
-------------------
113.0 119.0
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
169.0 172.0
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
0.0 78.0
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
90.0 110.0
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
120.0 168.0
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command succ

In [34]:
fight_vid = glob.glob(fight_path + '**/*.mp4', recursive=True)

In [35]:
import cv2

In [36]:
len(fight_vid)
import shutil

In [37]:
def get_random_timesplit(s,e):
    time_arr = []
    tmp = 0
    while tmp<e:
        ran = np.random.rand() * 3 + 2
        tmp += ran
        time_arr.append([s,tmp])
        s += ran
        
    return time_arr[:-1]

In [41]:
for fight in fight_vid:
    cap = cv2.VideoCapture(fight)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cropped = c_fight_path + os.path.basename(fight).split('.mp4')[0]
    print(fight, int(frame_count/fps))
    if frame_count / fps < 7:
        #print(int(frame_count/fps),' copy ', cropped)
        shutil.copy(fight,cropped +'.mp4')
    else:
        timesplit = get_random_timesplit(0,frame_count/fps)
        index = 1
        for tt in timesplit:
            start = tt[0]
            end = tt[1]
            print(start,end,cropped + '_' + str(index) + '.mp4')
            ffmpeg_extract_subclip(fight,int(start),int(end),cropped + '_' + str(index) + '.mp4')
            index+=1
    print('------')
        

C:/Users/Seogki/Downloads/UCF_Cropped/fight\AA-V-UCF-Abuse001_x264_1.mp4 11
0 3.243772953867633 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_fight/AA-V-UCF-Abuse001_x264_1_1.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
3.243772953867633 6.44991201924619 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_fight/AA-V-UCF-Abuse001_x264_1_2.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
------
C:/Users/Seogki/Downloads/UCF_Cropped/fight\AA-V-UCF-Abuse004_x264_1.mp4 9
0 2.2476081674867476 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_fight/AA-V-UCF-Abuse004_x264_1_1.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
2.2476081674867476 5.0521886888213885 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_fight/AA-V-UCF-Abuse004_x264_1_2.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
5.0521886888213885 8.636085616489206 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_fight/AA-V-UCF-Abuse004_x264_1_3.mp4


Moviepy - Command successful
18.28413648360617 21.332505681026554 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_fight/AA-V-UCF-Abuse039_x264_1_7.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
21.332505681026554 23.519708339902127 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_fight/AA-V-UCF-Abuse039_x264_1_8.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
23.519708339902127 28.0809260105325 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_fight/AA-V-UCF-Abuse039_x264_1_9.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
28.0809260105325 30.862097946451723 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_fight/AA-V-UCF-Abuse039_x264_1_10.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
------
C:/Users/Seogki/Downloads/UCF_Cropped/fight\AA-V-UCF-Abuse040_x264_1.mp4 7
0 3.3833350426213604 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_fight/AA-V-UCF-Abuse040_x264_1_1.mp4
Moviepy - Running:
>>> "+ " "

Moviepy - Command successful
30.023813116242746 32.31882473744432 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_fight/AA-V-UCF-Abuse050_x264_4_9.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
32.31882473744432 35.123886258268925 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_fight/AA-V-UCF-Abuse050_x264_4_10.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
35.123886258268925 37.655246121274146 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_fight/AA-V-UCF-Abuse050_x264_4_11.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
37.655246121274146 40.902414652778845 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_fight/AA-V-UCF-Abuse050_x264_4_12.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
40.902414652778845 45.64298801946795 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_fight/AA-V-UCF-Abuse050_x264_4_13.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
------
C:/Users/Seog

Moviepy - Command successful
3.608294561411495 8.298112143638825 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_fight/AA-V-UCF-Assault005_x264_1_2.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
8.298112143638825 12.816843552255548 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_fight/AA-V-UCF-Assault005_x264_1_3.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
------
C:/Users/Seogki/Downloads/UCF_Cropped/fight\AA-V-UCF-Assault006_x264_1.mp4 13
0 2.2215096337965905 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_fight/AA-V-UCF-Assault006_x264_1_1.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
2.2215096337965905 6.421116754790003 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_fight/AA-V-UCF-Assault006_x264_1_2.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
6.421116754790003 9.584412183675685 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_fight/AA-V-UCF-Assault006_x264_1_3.mp4
Moviepy - Running:

Moviepy - Command successful
3.810157150903263 7.955089536101298 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_fight/AA-V-UCF-Assault015_x264_2_2.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
------
C:/Users/Seogki/Downloads/UCF_Cropped/fight\AA-V-UCF-Assault016_x264_1.mp4 10
0 2.074231994342479 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_fight/AA-V-UCF-Assault016_x264_1_1.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
2.074231994342479 5.780720054438705 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_fight/AA-V-UCF-Assault016_x264_1_2.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
5.780720054438705 8.95976581895976 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_fight/AA-V-UCF-Assault016_x264_1_3.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
------
C:/Users/Seogki/Downloads/UCF_Cropped/fight\AA-V-UCF-Assault016_x264_2.mp4 6
------
C:/Users/Seogki/Downloads/UCF_Cropped/fight\AA-V-UCF

Moviepy - Command successful
5.492451903313675 7.606841620469039 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_fight/AA-V-UCF-Assault036_x264_2_3.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
------
C:/Users/Seogki/Downloads/UCF_Cropped/fight\AA-V-UCF-Assault036_x264_3.mp4 19
0 2.8629213652023893 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_fight/AA-V-UCF-Assault036_x264_3_1.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
2.8629213652023893 5.9372090749001005 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_fight/AA-V-UCF-Assault036_x264_3_2.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
5.9372090749001005 9.635355353103616 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_fight/AA-V-UCF-Assault036_x264_3_3.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
9.635355353103616 11.762896994432703 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_fight/AA-V-UCF-Assault036_x264_3_4.mp4
Moviepy - Runnin

Moviepy - Command successful
2.1268545291438072 5.570203628321963 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_fight/AA-V-UCF-Assault044_x264_1_2.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
5.570203628321963 9.057633449371293 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_fight/AA-V-UCF-Assault044_x264_1_3.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
9.057633449371293 13.51851187849018 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_fight/AA-V-UCF-Assault044_x264_1_4.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
------
C:/Users/Seogki/Downloads/UCF_Cropped/fight\AA-V-UCF-Assault045_x264_1.mp4 4
------
C:/Users/Seogki/Downloads/UCF_Cropped/fight\AA-V-UCF-Assault046_x264_1.mp4 5
------
C:/Users/Seogki/Downloads/UCF_Cropped/fight\AA-V-UCF-Assault047_x264_1.mp4 21
0 4.0630494464591 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_fight/AA-V-UCF-Assault047_x264_1_1.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Movi

In [42]:
nofight_vid = glob.glob(nofight_path + '**/*.mp4', recursive=True)
for fight in nofight_vid:
    cap = cv2.VideoCapture(fight)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cropped = c_nofight_path + os.path.basename(fight).split('.mp4')[0]
    print(fight, int(frame_count/fps))
    if frame_count / fps < 7:
        #print(int(frame_count/fps),' copy ', cropped)
        shutil.copy(fight,cropped+'.mp4')
    else:
        timesplit = get_random_timesplit(0,frame_count/fps)
        index = 1
        for tt in timesplit:
            start = tt[0]
            end = tt[1]
            if end-start < 1:
                break
            print(start,end,cropped + '_' + str(index) + '.mp4')
            ffmpeg_extract_subclip(fight,int(start),int(end),cropped + '_' + str(index) + '.mp4')
            index+=1
    print('------')
        

C:/Users/Seogki/Downloads/UCF_Cropped/noFight\AA-N-UCF-Abuse001_x264_1.mp4 8
0 4.903344155766235 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/AA-N-UCF-Abuse001_x264_1_1.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
4.903344155766235 7.767586529290128 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/AA-N-UCF-Abuse001_x264_1_2.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
------
C:/Users/Seogki/Downloads/UCF_Cropped/noFight\AA-N-UCF-Abuse001_x264_2.mp4 16
0 4.527425404530091 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/AA-N-UCF-Abuse001_x264_2_1.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
4.527425404530091 8.161649559792568 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/AA-N-UCF-Abuse001_x264_2_2.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
8.161649559792568 12.788167360495734 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/AA-N-UCF-Abuse001_x

Moviepy - Command successful
2.3489889276249922 7.175214905459341 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/AA-N-UCF-Abuse004_x264_4_2.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
7.175214905459341 11.01750238247978 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/AA-N-UCF-Abuse004_x264_4_3.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
11.01750238247978 14.789642382928095 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/AA-N-UCF-Abuse004_x264_4_4.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
14.789642382928095 18.086906818061202 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/AA-N-UCF-Abuse004_x264_4_5.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
18.086906818061202 21.963758302441803 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/AA-N-UCF-Abuse004_x264_4_6.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
21.963758302441

Moviepy - Command successful
3.188618780312342 5.8650492066433335 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/AA-N-UCF-Abuse008_x264_2_2.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
5.8650492066433335 10.58788842975434 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/AA-N-UCF-Abuse008_x264_2_3.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
10.58788842975434 13.41389314772207 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/AA-N-UCF-Abuse008_x264_2_4.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
13.41389314772207 16.66334917808253 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/AA-N-UCF-Abuse008_x264_2_5.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
16.66334917808253 20.19673713645379 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/AA-N-UCF-Abuse008_x264_2_6.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
20.19673713645379 2

Moviepy - Command successful
21.469178566627455 23.868925312904484 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/AA-N-UCF-Abuse008_x264_4_7.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
23.868925312904484 27.353212443387523 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/AA-N-UCF-Abuse008_x264_4_8.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
27.353212443387523 31.716232800655398 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/AA-N-UCF-Abuse008_x264_4_9.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
31.716232800655398 35.01470175453682 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/AA-N-UCF-Abuse008_x264_4_10.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
35.01470175453682 39.62233814252848 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/AA-N-UCF-Abuse008_x264_4_11.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
39.622338142

Moviepy - Command successful
35.23665801544398 39.59381391053118 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/AA-N-UCF-Abuse018_x264_1_11.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
39.59381391053118 43.35936799529225 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/AA-N-UCF-Abuse018_x264_1_12.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
43.35936799529225 46.45300901909805 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/AA-N-UCF-Abuse018_x264_1_13.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
46.45300901909805 49.328078343840005 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/AA-N-UCF-Abuse018_x264_1_14.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
49.328078343840005 53.858160704326146 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/AA-N-UCF-Abuse018_x264_1_15.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
------
C:/Use

...

Moviepy - Command successful
34.40373522044543 36.5155828135354 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/AA-N-UCF-Assault027_x264_4_11.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
36.5155828135354 39.14879487983652 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/AA-N-UCF-Assault027_x264_4_12.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
39.14879487983652 43.3901883035119 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/AA-N-UCF-Assault027_x264_4_13.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
43.3901883035119 47.88275934805007 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/AA-N-UCF-Assault027_x264_4_14.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
47.88275934805007 52.38136674388186 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/AA-N-UCF-Assault027_x264_4_15.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
52.3813667

Moviepy - Command successful
38.84595035482103 43.11561367319544 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/AA-N-UCF-Assault034_x264_1_13.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
43.11561367319544 48.02774206239647 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/AA-N-UCF-Assault034_x264_1_14.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
------
C:/Users/Seogki/Downloads/UCF_Cropped/noFight\AA-N-UCF-Assault035_x264_1.mp4 2
------
C:/Users/Seogki/Downloads/UCF_Cropped/noFight\AA-N-UCF-Assault037_x264_1.mp4 58
0 2.354082434718609 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/AA-N-UCF-Assault037_x264_1_1.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
2.354082434718609 5.316823037635022 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/AA-N-UCF-Assault037_x264_1_2.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
5.316823037635022 9.352028414614445 C:/Users

Moviepy - Command successful
2.773488002868251 6.607310590026127 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/AA-N-UCF-Assault042_x264_2_2.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
6.607310590026127 10.869345932095987 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/AA-N-UCF-Assault042_x264_2_3.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
------
C:/Users/Seogki/Downloads/UCF_Cropped/noFight\AA-N-UCF-Assault042_x264_3.mp4 8
0 3.667480973579109 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/AA-N-UCF-Assault042_x264_3_1.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
------
C:/Users/Seogki/Downloads/UCF_Cropped/noFight\AA-N-UCF-Assault042_x264_4.mp4 207
0 4.146635830693196 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/AA-N-UCF-Assault042_x264_4_1.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
4.146635830693196 7.742943080369576 C:/Users/Seogki/Download

Moviepy - Command successful
139.30354897309442 143.51183171985497 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/AA-N-UCF-Assault042_x264_4_40.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
143.51183171985497 148.12012816300705 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/AA-N-UCF-Assault042_x264_4_41.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
148.12012816300705 151.6577844777042 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/AA-N-UCF-Assault042_x264_4_42.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
151.6577844777042 155.57718035016276 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/AA-N-UCF-Assault042_x264_4_43.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
155.57718035016276 157.91741576793373 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/AA-N-UCF-Assault042_x264_4_44.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successfu

Moviepy - Command successful
50.328229715226676 52.51950357542091 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/AA-N-UCF-Assault047_x264_1_15.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
52.51950357542091 54.76146966727227 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/AA-N-UCF-Assault047_x264_1_16.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
54.76146966727227 57.93555823406367 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/AA-N-UCF-Assault047_x264_1_17.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
------
C:/Users/Seogki/Downloads/UCF_Cropped/noFight\AA-N-UCF-Assault048_x264_1.mp4 5
------
C:/Users/Seogki/Downloads/UCF_Cropped/noFight\AA-N-UCF-Assault048_x264_2.mp4 51
0 3.498122177266914 C:/Users/Seogki/Downloads/UCF_Cropped/cropped_noFight/AA-N-UCF-Assault048_x264_2_1.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
3.498122177266914 5.936361722748249 C:/Use

In [28]:
import cv2

In [29]:
cap = cv2.VideoCapture('C:/Users/Seogki/Downloads/UCF_Cropped/fight/AV-V-UCF-Abuse001_x264_1.mp4')

In [30]:
cnt = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    cnt += 1
print(cnt)

90


In [ ]:
c